In [1]:
import sys
import os
from pathlib import Path
import time

root_dir = Path(os.getcwd()).resolve().parent
sys.path.append(str(root_dir))

import torch
import torch.nn as nn
import torchvision.transforms as transforms

from models import tiramisu_nclasses
from datasets import joint_transforms
import src.training as train_utils
from src.experiment import Experiment

In [2]:
crop_size =  60 # for teting purposes
batch_size = 4

LR = 0.0001
LR_DECAY= 0.995
beta1= 0.9
beta2=  0.999
weight_decay= 0.001
factor =  0.5
patience =  5

In [3]:
from datasets.rescuenet_resized import Dataset

train_joint_transformer = transforms.Compose([
    joint_transforms.JointRandomCrop(crop_size),
    joint_transforms.JointRandomHorizontalFlip()
    ])

dataset = Dataset(split='train', scale=713, joint_transform=train_joint_transformer)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True)

num_classes = len(dataset.class_names)

In [4]:
torch.cuda.manual_seed(0)
device = torch.device("cuda") 
model = tiramisu_nclasses.FCDenseNet103(len(dataset.class_names))
model.cuda()


FCDenseNet(
  (firstconv): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (denseBlocksDown): ModuleList(
    (0): DenseBlock(
      (layers): ModuleList(
        (0): DenseLayer(
          (norm): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv): Conv2d(48, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d(p=0.2, inplace=False)
        )
        (1): DenseLayer(
          (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (drop): Dropout2d(p=0.2, inplace=False)
        )
        (2): DenseLayer(
          (norm): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv): Conv2d(80, 16, kernel_size=(3, 3), strid

In [5]:
print('initializing weights')
model.apply(train_utils.weights_init)
START_EPOCH = 0

initializing weights


In [6]:
model = nn.DataParallel(model, device_ids=[0,1])

In [7]:

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=LR, 
                             betas=(beta1, beta2),
                             eps=1e-08, 
                             weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=patience)


In [8]:

fw = train_utils.inverse_frequency_weights(dataset.pixel_count)
class_weights =  torch.log(fw).cuda()

In [9]:
criterion = nn.NLLLoss(weight=class_weights)

### chose the number of epochs

In [10]:
END_EPOCH = 1

In [11]:

since = time.time()

for epoch in range(0, END_EPOCH):
    print('Starting Training Epoch:', epoch)
    # Train ###
    trn_loss, trn_err, iou_result = train_utils.train(
        model, dataloader, num_classes, optimizer, criterion, device)

    print('Epoch {:d}\nTrain - Loss: {:.4f}, Acc: {:.4f}'.format(
        epoch, trn_loss, 1-trn_err))    
    time_elapsed = time.time() - since  
    print('Train Time {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    # updating LR
    scheduler.step(trn_loss)  

Starting Training Epoch: 0


Epoch 0
Train - Loss: 0.0329, Acc: 0.9908
Train Time 0m 8s
